# **redis-feast-gcp**: 00 - Feature Store Intro

Explore the setup and use of a Feature Store, deployed in GCP, and backed by Redis as the online store component.

**This notebook assumes that you've already set up your feature store and model repo in GCP**

![architecture](img/redis-feast-gcp-architecture.png)

## Install Dependencies

Pip install the feature_store package to use the provided tools.

In [ ]:
!pip install pandas "feast[gcp, redis]==0.22.0" matplotlib

In [ ]:
# Automatically restart kernel after installs
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# Fetching Historical Data

The Feast SDK allows us to pull historical data to generate a training dataset. In this cae, the data comes from BigQuery tables.

To make things simpler, we use the [`DataFetcher`](utils/data_fetcher.py) class that wraps Feast, which wraps Redis and BigQuery.

In [ ]:
from feature_store.repo import config
from feature_store.utils import (
    DataFetcher,
    storage
)

# Load Feast feature store
fs = storage.get_feature_store(
    config_path=config.REPO_CONFIG,
    bucket_name=config.BUCKET_NAME
)

# Load data fetcher utility class
data_fetcher = DataFetcher(fs)

In [ ]:
# The data fetcher class contains the list of input features to use for model training
print(data_fetcher.X_cols)

print(data_fetcher.y_col)

In [ ]:
# Fetch historical training data with Feast
ds = data_fetcher.get_training_data(
    entity_query=f"""
        select
            state,
            date as event_timestamp
        from
            {config.BIGQUERY_DATASET_NAME}.{config.WEEKLY_VACCINATIONS_TABLE}
    """
)

# Clean up any nulls
ds.dropna(inplace=True)
ds.sort_values(['event_timestamp', 'state'], axis=0, inplace=True)

In [ ]:
# Subset the data to the state of Virginia (in the US)
virginia = ds[ds.state == 'Virginia']

In [ ]:
virginia.head()

In [ ]:
virginia.tail()

In [ ]:
virginia.weekly_vaccinations_count.describe()

In [ ]:
# Investigate trends across features for prediction weekly vaccinations count!
virginia.plot.scatter(y='weekly_vaccinations_count', x='lag_2_vaccine_interest')

## Low latency feature retrieval

To make a prediction in real-time, we need to do the following:

1. Fetch our features from Redis using the Feast client
2. Pass these features to our model
3. Return these predictions to the user

Here is an example pulling the latest features data from Redis (online store) for the state of Virginia

In [ ]:
# Fetch online feature
online_feature = data_fetcher.get_online_data(state="Virginia")

In [ ]:
online_feature